In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE 
import cv2
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import savez_compressed
import matplotlib.pyplot as plt 
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from numpy import asarray

from pydicom import dcmread

import glob 
from os import listdir
from numpy import asarray
from numpy import vstack

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from numpy import savez_compressed
import numpy as np 



In [ ]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from matplotlib import pyplot

In [ ]:
test = os.path.join("../input/unetdata/testing_256.npz")
training = os.path.join("../input/input/unetdata/training_256.npz")

In [ ]:
training

In [ ]:
def plotMask(X,y):
    sample = []
    
    for i in range(6):
        left = X[i]
        right = y[i]
        combined = np.hstack((left,right))
        sample.append(combined)
        
        
    for i in range(0,6,3):

        plt.figure(figsize=(25,10))
        
        plt.subplot(2,3,1+i)
        plt.imshow(sample[i].astype('uint8'))
        
        plt.subplot(2,3,2+i)
        plt.imshow(sample[i+1].astype('uint8'))
        
        
        plt.subplot(2,3,3+i)
        plt.imshow(sample[i+2].astype('uint8'))
        
        plt.show()

# plot training and testing images 

In [ ]:
# Load training and testing data
datatrain = load("../input/unetdata/training_256.npz")
X_train, y_train = datatrain['arr_0'], datatrain['arr_1']
plotMask(X_train, y_train)

In [ ]:
datatest = load("../input/unetdata/testing_256.npz")
X_test, y_test = datatest['arr_0'], datatest['arr_1']
plotMask(X_test, y_test)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
assert X_train.shape == y_train.shape
assert X_test.shape == y_test.shape

In [ ]:

assert X_train.shape == y_train.shape
assert X_test.shape == y_test.shape
images = np.concatenate((X_train,X_test),axis=0)
mask  = np.concatenate((y_train,y_test),axis=0)

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler


def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet(input_size=(256,256,1)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', dtype='float32')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(3, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])

In [ ]:
model = unet(input_size=(256,256,3))
model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss,
                  metrics=[dice_coef, 'binary_accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('cxr_reg')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                   patience=3,
                                   verbose=1, mode='min', epsilon=0.0002, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
from IPython.display import clear_output
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
model.compile(optimizer=Adam(lr=0.00001), 
              loss=[dice_coef_loss], 
           metrics = [dice_coef, 'binary_accuracy', 'AUC'])

train_vol, validation_vol, train_seg, validation_seg = train_test_split((images)/255, 
                                                            (mask>127).astype(np.float32), 
                                                            test_size = 0.1,random_state = 2018)

train_vol, test_vol, train_seg, test_seg = train_test_split(train_vol,train_seg, 
                                                            test_size = 0.1, 
                                                            random_state = 2018)



In [ ]:
plt.imshow(test_vol[0].astype(np.float32))

In [ ]:
loss_history = model.fit(x = train_vol,
                       y = train_seg,
                         batch_size = 8,
                  epochs = 120,
                  validation_data =(test_vol,test_seg) ,
                  callbacks=callbacks_list)

model.save('model.h5')

In [ ]:
model.save('model.h5')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
ax1.plot(loss_history.history['loss'], '-', label = 'Loss')
ax1.plot(loss_history.history['val_loss'], '-', label = 'Validation Loss')
ax1.legend()

ax2.plot(100*np.array(loss_history.history['binary_accuracy']), '-', 
         label = 'Accuracy')
ax2.plot(100*np.array(loss_history.history['val_binary_accuracy']), '-',
         label = 'Validation Accuracy')
ax2.legend()

In [ ]:
plt.plot(loss_history.history['loss'], '-', label = 'Loss')
plt.plot(loss_history.history['val_loss'], '-', label = 'Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Dice Loss')
plt.show()


In [ ]:
plt.plot(loss_history.history['auc'], '-', label = 'AUC')
plt.plot(loss_history.history['val_auc'], '-', label = 'Validation AUC')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.show()

In [ ]:
pred_candidates = np.random.randint(1,validation_vol.shape[0],10)
preds = model.predict(validation_vol)

In [ ]:
test_vol.shape[0]

In [ ]:
probs = model.predict(test_vol)

In [ ]:
pred_candidates

In [ ]:
preds.shape

In [ ]:
plt.imshow(preds[1])

In [ ]:


plt.figure(figsize=(20,10))

for i in range(0,9,4):
    plt.subplot(4,4,i+1)
    
    plt.imshow(validation_vol[pred_candidates[i]].astype(np.float32))
    plt.xlabel("Base Image")
    plt.axis('off')
    
    
    plt.subplot(4,4,i+2)
    plt.imshow(np.squeeze(validation_seg[pred_candidates[i]]).astype(np.float32))
    plt.xlabel("Mask")
    plt.axis('off')
    
    plt.subplot(4,4,i+3)
    plt.imshow(np.squeeze(preds[pred_candidates[i]]).astype(np.float32))
    plt.xlabel("Pridiction")
    plt.axis('off')
    
    plt.subplot(4,4,i+4)
    plt.imshow(validation_vol[pred_candidates[i]]+np.squeeze(preds[pred_candidates[i]]*0.2).astype(np.float32))
    plt.xlabel("Pridiction")
    plt.axis('off')
    
    plt.tight_layout()
    

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    fig = plt.figure(figsize = [8,8])
    for i in range(len(images)):
        # define subplot
        ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    fig.suptitle(a)
    fig.tight_layout()
    fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()

In [ ]:
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization


# for testing images

In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/testing_256.npz')

In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/testing_256.npz')
#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,65)
for i in range(20):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))
    
    

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('valid.csv')

# for training dataset 

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    #fig = plt.figure(figsize = [8,8])
    #for i in range(len(images)):
        # define subplot
        #ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        #pyplot.axis('off')
        # plot raw pixel data
        #ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        #ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    #fig.suptitle(a)
    #fig.tight_layout()
    #fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()
    
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization

In [ ]:
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization



In [ ]:
[X2, X1] = load_real_samples('../input/unetdata/training_256.npz')
#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,555-20)
for i in range(555-20):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('train.csv')

# final testing images

In [ ]:
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy.random import randint
import tensorflow.keras as tf
# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = ((X1) / 255.0)
    X2 = ((X2) / 255.0)
    return [X1, X2]
ssi = []
psi = []
iou = []
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    #fig = plt.figure(figsize = [8,8])
    #for i in range(len(images)):
        # define subplot
        #ax = pyplot.subplot(1, 3, 1 + i)
        #plt.figure(figsize = [5,5])
        # turn off axis
        #pyplot.axis('off')
        # plot raw pixel data
        #ax.imshow(np.squeeze(images[i]).astype(np.float32), cmap = 'gray')
        # show title
        #ax.set_title(titles[i])
    ss = str(float(ssim1))
    ps = str(float(psnr1))
    io = str(float(iou1))
    
    a =  'SSIM:' ,ss[:5], 'PSNR:' ,  ps[:5], 'IOU=', io[:5]
    print(a)
    
    ssi.append(ss)
    psi.append(ps)
    iou.append(io)
    #fig.suptitle(a)
    #fig.tight_layout()
    #fig.subplots_adjust(top= 1.5)
    #plt.savefig(name, bbox_inches = 'tight', pad_inches = 0)
    
    pyplot.show()
    
import tensorflow as tf 
#tf.random.Generator = None  # Patch for a bug
from tensorflow_addons.layers import InstanceNormalization

In [ ]:
[X2, X1] = load_real_samples('../input/finaldata/finaltesting.npz')
#print('Loaded', X1.shape, X2.shape)
# load model

#print('Loaded', X1.shape, X2.shape)
# load model


import numpy as np 
XD = np.arange(0,555-20)
for i in range(555-20):
    print('for image number ', i )
    name = 'mname'+ '_'+str(i) +'_' + '.png'

    ix = XD[i]
    ix = np.array([ix])
    tar_image, src_image   = X1[[ix]], X2[[ix]]
    # generate image from source
    gen_image = model.predict(src_image)
    # plot all three images
    images = vstack((src_image, gen_image, tar_image))
    images = (images + 1) / 2.0
    df = images[1]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (df.min(), df.max()))
    df1 = ((df) / df.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (df1.min(), df1.max()))
    dfn = images[2]

    print('RAW image: ', 'Min: %.3f, Max: %.3f' % (dfn.min(), dfn.max()))
    dfn1 = ((dfn) / dfn.max())*255
    print('After rescaling: Min: %.3f, Max: %.3f' % (dfn1.min(), dfn1.max()))

    a =tf.convert_to_tensor(df1, tf.uint8)
    b = tf.convert_to_tensor(dfn1, tf.uint8)
    ssim1 = tf.image.ssim(a,b, max_val=255, filter_size=11,filter_sigma=1.5, k1=0.01, k2=0.03) 
    psnr1 = tf.image.psnr(a, b, max_val=255)
    print(float(ssim1))
    print(float(psnr1))
    im1 = tf.image.convert_image_dtype(a, tf.float32)
    im2 = tf.image.convert_image_dtype(b, tf.float32)
    ssim2 = tf.image.ssim(im1,im2, max_val=1, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    # IOU calculation 
    a1 = a[:,:,0]
    img = a1/np.max(a1)
    imgn = np.where(img<0.9,0, 1)

    #plt.imshow(imgn, cmap = 'gray')

    b1 = b[:,:,0]
    imgb = b1/np.max(b1)
    imgb = np.where(imgb<0.9,0, 1)
    #plt.imshow(imgb, cmap = 'gray')

    m = tf.keras.metrics.MeanIoU(num_classes=2)
    m.update_state(imgn, imgb)
    iou1 = m.result().numpy()
    plot_images(src_image, gen_image, tar_image)

In [ ]:
iou1 = []
ssim1 = []
PSNR = []
for i in range(len(iou)):
    iou1.append(float(iou[i]))
    ssim1.append(float(ssi[i]))
    PSNR.append(float(psi[i]))

In [ ]:
import numpy as np 
print('mean iou: ', np.mean(iou1))
print('mean SSIM: ', np.mean(ssim1))
print('mean PSNR: ', np.mean(PSNR))
print('mean iou: ', np.std(iou1))
print('mean SSIM: ', np.std(ssim1))
print('mean PSNR: ', np.std(PSNR))

In [ ]:
import seaborn as sns
sns.set()
plt.plot(iou1)
plt.ylabel('IOU')
plt.xlabel('Image id')
plt.show()
plt.plot(ssim1)
plt.ylabel('SSIM')
plt.xlabel('Image id')
plt.show()
plt.plot(PSNR)
plt.ylabel('PSNR')
plt.xlabel('Image id')
plt.show()

In [ ]:
df = (pd.DataFrame([iou1, ssim1, PSNR])).transpose()
df.columns = ['iou', 'ssim', 'PSNR']
df.to_csv('testfinal.csv')